# Reconnaissance Optique de Caractères avec Keras

**Source :** Cours de Franck Bardol, [LinkedIn Learning](https://www.linkedin.com/learning/decouvrir-le-deep-learning-avec-keras/bienvenue-dans-le-deep-learning-avec-keras?autoplay=true).

## OCR : Optical Caracter Recognition

In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
@author: franck BARDOL
"""


## Objectif : 
Nous allons voir comment apprendre à reconnaître des caractères optiques avec Keras.

Il s'agit de reconnître des chiffres traçés à la main.

Ce type d'application est déployé pour la lecture optique des chèques bancaires par exemple.

Nous allons écrire un **Réseau de neurones à Convolution** afin de reconnaître les caractères manuscrits. 


Le réseau de neurones devra apprendre à lire les chiffres. C'est-à-dire, prédire correctement la valeur du chiffre tracé.


Cela n'a rien d'évident. 
Là où un humain voit une image représentant un chiffre, le réseau ne voit, pour sa part, qu'un ensemble de pixels

## Importation des librairies nécessaires

In [ ]:
# ==========================================
#                   MNIST data set
# ==========================================

from keras.datasets import mnist
from keras import models
from keras import layers
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as pl

## Le data-set 
On importe les images du [MNIST](https://en.wikipedia.org/wiki/MNIST_database).
C'est un ensemble de 60,000 images de chiffres en N&B 

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print("nb images = ",train_images.shape[0])
print("unique label = ",np.unique(train_labels))

### Préparation des données
On doit "re-tailler" les images dans un format standard.

Le format attendu est une image de taille 28 pixel sur 28 pixel.

Les images sont Noir & Blanc. On ajoute une dimension supplémentaire pour tenir compte de cette information.

In [ ]:
print("dimension data set :" , train_images.shape)

### Visualisation de quelques images
Avant de se lancer dans l'écriture d'un modèle de Deep Learning, il est toujours intéressant de **visualiser** les données.

Pour cela, on utilise la librairie *matplotlib*

Lien internet [matplotlib](https://matplotlib.org/)

In [ ]:
# choix du numéro de l'image 
num_img = 100
pl.figure()
pl.imshow(train_images[num_img,:,:], cmap = pl.get_cmap('gray'))
pl.show()

# quel chiffre ?
print(train_labels[num_img])
num_label = train_labels[num_img]
print("c est un : " , num_label)

### Redimension - Normalisation des images
* On re-dimensionne (*reshape*) pour ajouter **une** dimension.
Si les images étaient en couleur, on devrait alors ajouter **trois** dimensions (RGB : red - green - blue).
* On normalise les images d'origine qui sont encodées sur 255 niveaux de gris. 
Les valeurs des pixels seront donc comprises entre 0 et 1

In [ ]:
# re-dimension :
train_images = train_images.reshape((60000, 28, 28, 1))
# normalisation :
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

### One hot encoding des labels 
Les outputs (labels) sont une étiquette (comprise entre 0 et 9) qui représente la valeur du chiffre traçé.
C'est la sortie que le réseau devra apprendre à reconnaitre.

Le *One Hot encoding* est un traitement classique. 

Il consiste à étendre la **dimension** de la sortie.
On passe d'un nombre entre 0 et 9 à un vecteur (tableau) de dimension 9.


Exemple de One Hot Encoding sur des nombres compris entre 0 et 2 : 

0 -> {1 , 0 , 0} 

1 -> {0 , 1 , 0} 

2 -> {0 , 0 , 1} 

Ce traitement facilite l'apprentissage du réseau


In [ ]:
train_labels.shape

In [ ]:
# One Hot Encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## Deep Learning : Convolutionnal Neural Network avec Keras

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))

### Couche  de traitement *flatten* 
Aprés les couches de **convolution**, on doit **toujours** ajouter une couche de traitement **Flatten**.

Cette couche met à plat les structures en 2D issues de la convolution.

La couche *Flatten*  assure le passage 2D -> 1D

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### remarque :
l'apprentissage du modèle est lent. 
Vous pouvez l'accélérer en déportant les calculs sur GPU

In [ ]:
history = model.fit(train_images, 
          train_labels, 
          validation_split = 0.1,
          epochs = 10, 
          batch_size = 500)

In [ ]:
pl.figure()
pl.plot(history.history['loss'])
pl.plot(history.history['val_loss'])
pl.legend(["training","test"])

### Exercice
Observez la courbe d'apprentissage.

L'apprentissage s'est-il déroulé correctement selon vous ?

## Performance du modèle 
Accuracy : % d'instances bien classées

In [ ]:
# Evaluation
print(model.evaluate(test_images, test_labels))

In [ ]:
np.argmax(test_labels , axis = 1)

## Matrice de Confusion 

Partie avancée. Cette partie peut être passée en 1ère lecture ...

Nous utilisons une matrice de confusion pour interpréter les résultats.

La matrice de confusion est un matrice *carrée* comportant autant de lignes qu'il y a de classes dans le problème.

Ici, nous avons **dix** classes correspondant aux chiffres de 0 à 9. Il y aura donc **dix** lignes et **dix** colonnes. 
Dans notre cas, c'est une matrice 10 x 10

En colonne, on reporte les valeurs **prédites** par l'algorithme. 
("*predicted*" dans le tableau ci-dessous).

En ligne, on reporte les valeurs **observées** dans la réalité.
("*True*" dans le tableau ci-dessous).

On reporte dans le tableau les instances de la manière suivante :

> - dans la diagonale principale du tableau le nombre d'instances correctement prédites.  
Les vrais-négatifs (*true negative* : `TN`) et les vrais-positifs (*true positive* : `TP`).
> - dans les cellules restantes, on reporte les erreurs de classification : les faux-positifs (`FP`) et les faux-négatifs (`FN`) 

Pour aller plus loin sur la matrice de confusion :
https://fr.wikipedia.org/wiki/Matrice_de_confusion

Affichage d'une matrice de confusion :

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

y_pred = model.predict(test_images)
# pick the best predicted (max)
y_pred = np.argmax(y_pred , axis = 1)

### Affichage non formaté de la matrice 

In [ ]:
# les labels (etiquettes)
lbls = np.argmax(test_labels , axis = 1)
conf_mat = confusion_matrix(lbls  , y_pred ,labels = [0,1,2,3,4,5,6,7,8,9])
print(conf_mat)

### Affichage formaté de la matrice

In [ ]:
cm_display = ConfusionMatrixDisplay(conf_mat,[0,1,2,3,4,5,6,7,8,9])
cm_display.plot()

### Analyse de la plus grande erreur

In [ ]:
#  ignorer la diagonale de la matrice de confusion
mask = np.ones(conf_mat.shape, dtype=bool)
np.fill_diagonal(mask, 0)
# pick max -> instances mal classfiées
max_value = conf_mat[mask].max()

position = np.where(conf_mat == max_value)

In [ ]:
print("plus grand nombre d'erreur : " , max_value)
print("position du plus grand nombre d'erreur : ligne {}, colonne {}".format(position[0] , position[1]))
print("on confond les {} (vrai label) avec les {} (label prédit)".format(position[0] , position[1]))

### Affichage de toutes les plus grandes erreurs

In [ ]:
# intersection valeur prédite , vraie valeur -> position des plus grandes erreurs
s_predict = set(np.where(y_pred == position[1])[0])
s_true = set( np.where(lbls == position[0])[0])
# on stocke les positions des erreurs 
idx_err = s_predict.intersection(s_true)

In [ ]:
# retour au format d'origine pour affichage 
tst_img = test_images.reshape((10000, 28, 28))
# afficher petites vignettes 
pl.rcParams["figure.figsize"] = (1,1)

In [ ]:
for item in idx_err:
  # choix du numéro de l'image 
  num_img = item
  pl.figure()
  pl.imshow(tst_img[num_img,:,:], cmap = pl.get_cmap())
  pl.show()

  # quel chiffre ?
  num_label = test_labels[num_img]
  print("vrai label : {}, prediction : {}".format(np.argmax(num_label) , y_pred[item]))